# Clustering - Conjunto de Datos Iris

Aplicando técnicas de clustering se buscó segmentar las especies de la flor Iris utilizando el conjunto de datos clásico de Machine Learning. Se explorarán los datos visualmente y se implementará el algoritmo K-Means para agrupar las muestras según la forma del sépalo y el pétalo. Además, se empleará el método del "codo" y la librería kneed para determinar el número óptimo de clusters. Finalmente, se compararán los resultados obtenidos con los datos reales para evaluar la efectividad del modelo.

## Sección 1

1. **Visualicen los datos para ver si pueden detectar algunos grupos. Ayuda: utilicen la forma del sépalo:**

2. **Creen 2 "clusters" utilizando K_Means Clustering y grafiquen los resultados.**


3. **Estandaricen los datos e intenten el paso 2, de nuevo. ¿Qué diferencias hay, si es que lo hay?**


4. **Utilicen el método del "codo" para determinar cuantos "clusters" es el ideal. (prueben un rango de 1 a 10)**

5. **Basado en la gráfica del "codo" realicen varias gráficas con el número de clusters (unos 3 o 4 diferentes) que Uds creen mejor se ajusten a los datos.**

6. **Comparen sus soluciones con los datos reales, archivo: iris-con-respuestas.csv**

**Obviamente solo hay tres especies, porque ese es el archivo de datos reales!**

**¿Funcionó el clustering con la forma del sépalo?****